## 『本次練習內容』
#### 學習如何搭建 Residual Block
####  學習如何搭建Inception-ResNet中的 Inception Block

## 『本次練習目的』
  #### 了解 Residual Block原理
  #### 了解如何結合Inception 與 Residual概念

---

## Part1

In [1]:
from keras import backend as K
from keras import layers
from keras.models import Model
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Input
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import GlobalMaxPooling2D
from keras.layers import GlobalAveragePooling2D
from keras.layers import BatchNormalization
from keras.layers import Activation
from keras.layers import Concatenate
from keras.layers import Lambda

import numpy as np
from google.colab.patches import cv2_imshow
import cv2

Using TensorFlow backend.


![ResNet_Structure](https://cvdl.cupoy.com/HomeworkAction.do?op=getHomeworkFileContent&hwid=D20&filepath=ResNet_Structure.png)

## ResNetV1

In [0]:
# ResNetV1

def Residual_block(input_tensor, kernel_size, filters, stage, block):
    filters1, filters2, filters3 = filters
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    x = Conv2D(filters1, (1, 1), name=conv_name_base + '2a')(input_tensor)
    x = BatchNormalization(axis=3, name=bn_name_base + '2a')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters2, kernel_size,
               padding='same', name=conv_name_base + '2b')(x)
    x = BatchNormalization(axis=3, name=bn_name_base + '2b')(x)
   

    x = layers.add([x, input_tensor])
    x = Activation('relu')(x)
    return x

## 參考ResNetV1 搭建 ResNetV2版本的Residual Block

In [0]:
# ResNetV2

def Residual_block_v2(input_tensor, kernel_size, filters, stage, block):
    filters1, filters2, filters3 = filters
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    x = Conv2D(filters1, (1, 1), name=conv_name_base + '2a')(input_tensor)
    x = BatchNormalization(axis=3, name=bn_name_base + '2a')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters2, kernel_size,
               padding='same', name=conv_name_base + '2b')(x)

    x = layers.add([x, input_tensor])
    x = Activation('relu')(x)
    return x

## 試試看自己設計一個先壓縮再回放的V2 Block

In [0]:
# ResNetV2s

def Residual_block_v2s(input_tensor, kernel_size, stage, block,reduce=96,ouput_size=128):

    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    x = Conv2D(reduce, (1, 1), name=conv_name_base + '2sa')(input_tensor)
    x = BatchNormalization(axis=3, name=bn_name_base + '2sa')(x)
    x = Activation('relu')(x)
    
    x = Conv2D(ouput_size, kernel_size,
               padding='same', name=conv_name_base + '2sb')(x)
    
    x = layers.add([x, input_tensor])

---

## Part2

In [0]:
def Conv2d_bn(x,filters,kernel_size,padding='same',strides=(1, 1),normalizer=True,activation='relu',name=None):
    if name is not None:
        conv_name = name + '_conv'
        bn_name = name + '_bn'
        act_name = name + '_act'
    else:
        conv_name = None
        bn_name = None
        act_name = None
    if K.image_data_format() == 'channels_first':
        bn_axis = 1
    else:
        bn_axis = 3
    x = Conv2D(
            filters, kernel_size,
            strides=strides, padding=padding,
            use_bias=False, name=conv_name)(x)
    if normalizer:
        x = BatchNormalization(axis=bn_axis, scale=False, name=bn_name)(x)
    if activation:
        x = Activation(activation, name=act_name)(x)
    return x

![Residual_block](https://img-blog.csdnimg.cn/20181216143902767.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L2JvaWNlX2dpcmw=,size_16,color_FFFFFF,t_70)

In [0]:
def Residual_block(input_tensor, kernel_size, filters, stage, block):
    filters1, filters2, filters3 = filters
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    x = Conv2D(filters1, (1, 1), name=conv_name_base + '2a')(input_tensor)
    x = BatchNormalization(axis=3, name=bn_name_base + '2a')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters2, kernel_size,
               padding='same', name=conv_name_base + '2b')(x)
    x = BatchNormalization(axis=3, name=bn_name_base + '2b')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters3, (1, 1), name=conv_name_base + '2c')(x)
    x = BatchNormalization(axis=3, name=bn_name_base + '2c')(x)

    x = layers.add([x, input_tensor])
    x = Activation('relu')(x)
    return x

## Incpetion Block-A

![Incpeiton](https://cvdl.cupoy.com/HomeworkAction.do?op=getHomeworkFileContent&hwid=D20&filepath=Inception-ResNet-A.png)

## Incpetion Block-B

![Incpeiton](https://cvdl.cupoy.com/HomeworkAction.do?op=getHomeworkFileContent&hwid=D20&filepath=Inception-ResNet-B.png)

## Incpetion Block-C

![Incpeiton](https://cvdl.cupoy.com/HomeworkAction.do?op=getHomeworkFileContent&hwid=D20&filepath=Inception-ResNet-C.png)

## 參考上方Residual_block搭建 Inception-ResNet中的Inception Block

In [0]:
def inception_resnet_block(x, scale, block_type, activation='relu'):
    '''scale: scaling factor to scale the residuals before adding them
            to the shortcut branch.(i.e., the output of passing `x` through an inception module)  
        Let `r` be the output from the residual branch,
            the output of this block will be `x + scale * r`.(簡單來說就是控制Residual branch的比例)'''

    if block_type == 'Incpetion_Block-A':
        branch_0 = Conv2d_bn(x, 32, (1,1), name=block_type + '2a_0')
        branch_1 = Conv2d_bn(x, 32, (1,1), name=block_type + '2b_0')
        branch_1 = Conv2d_bn(x, 32, (3, 3), name=block_type + '2b_1')
        branch_2 = Conv2d_bn(x, 32, (1, 1), name=block_type + '2c_0')
        branch_2 = Conv2d_bn(x, 48, (3, 3), name=block_type + '2c_1')
        branch_2 = Conv2d_bn(x, 64, (3, 3), name=block_type + '2c_2')
        branches = [branch_0, branch_1, branch_2]
    elif block_type == 'Incpetion_Block-B':
        branch_0 = Conv2d_bn(x, 192, (1, 1), name=block_type + '2a_0')
        branch_1 = Conv2d_bn(x, 128, (1, 1), name=block_type + '2b_0')
        branch_1 = Conv2d_bn(x, 160, (1, 7), name=block_type + '2b_1')
        branch_1 = Conv2d_bn(x, 192, (7, 1), name=block_type + '2b_2')
        branches = [branch_0, branch_1]
    elif block_type == 'Incpetion_Block-C':
        branch_0 = Conv2d_bn(x, 192, (1, 1), name=block_type + '2a_0')
        branch_1 = Conv2d_bn(x, 192, (1, 1), name=block_type + '2b_0')
        branch_1 = Conv2d_bn(x, 192, (1, 3), name=block_type + '2b_1')
        branch_1 = Conv2d_bn(x, 192, (3, 1), name=block_type + '2b_2')
        branches = [branch_0, branch_1]
    else:
        raise ValueError('Unknown Inception-ResNet block type. '
                         'Expects "block35", "block17" or "block8", '
                         'but got: ' + str(block_type))
    mixed = Concatenate(axis=3)(branches)
    
    '''確保輸入跟輸出深度相同'''
    up = Conv2d_bn(mixed,K.int_shape(x)[3],1,activation=None)
    
    '''導入殘差結構，並給予權重'''
    
    x = Lambda(lambda inputs, scale: inputs[0] + inputs[1] * scale, ##提示inputs[0]、inputs[1]
               output_shape=K.int_shape(x)[1:],
               arguments={'scale': scale},)([x,up])
    
    if activation is not None:
        x = Activation(activation)(x)
    return x


## 測試

In [8]:
img_input = Input(shape=(224,224,32))
xA=inception_resnet_block(img_input, 0.1, 'Incpetion_Block-A', activation='relu')
xB=inception_resnet_block(img_input, 0.1, 'Incpetion_Block-B', activation='relu')
xC=inception_resnet_block(img_input, 0.1, 'Incpetion_Block-C', activation='relu')

print(xA)
print(xB)
print(xC)












Tensor("activation_1/Relu:0", shape=(?, 224, 224, 32), dtype=float32)
Tensor("activation_2/Relu:0", shape=(?, 224, 224, 32), dtype=float32)
Tensor("activation_3/Relu:0", shape=(?, 224, 224, 32), dtype=float32)


## 嘗試導入Inception Block到 Vgg_Inception中

In [9]:
def VGG16_ResNet_Inception(include_top=True,input_tensor=None, input_shape=(224,224,1),
          pooling='max',classes=1000):
 
    img_input = Input(shape=input_shape)
    
  # Block 1
    x = Conv2d_bn(img_input,64, (3, 3), activation='relu', padding='same', name='block1_conv1')
    x = Conv2d_bn(x,64, (3, 3), activation='relu', padding='same', name='block1_conv2')
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

  # Block 2
    x = Conv2d_bn(x,128, (3, 3), activation='relu', padding='same', name='block2_conv1')
    x = Conv2d_bn(x,128, (3, 3), activation='relu', padding='same', name='block2_conv2')
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

  # Block 3
    x = inception_resnet_block(img_input, 0.1, 'Incpetion_Block-A', activation='relu')
    x = inception_resnet_block(img_input, 0.1, 'Incpetion_Block-B', activation='relu')
    x = inception_resnet_block(img_input, 0.1, 'Incpetion_Block-C', activation='relu')
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

  # Block 4
    x = Conv2d_bn(x,512, (3, 3), activation='relu', padding='same', name='block4_conv1')
    x = Conv2d_bn(x,512, (3, 3), activation='relu', padding='same', name='block4_conv2')
    x = Conv2d_bn(x,512, (3, 3), activation='relu', padding='same', name='block4_conv3')
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

  # Block 5 
    x = inception_resnet_block(img_input, 0.1, 'Incpetion_Block-A', activation='relu')
    x = inception_resnet_block(img_input, 0.1, 'Incpetion_Block-B', activation='relu')
    x = inception_resnet_block(img_input, 0.1, 'Incpetion_Block-C', activation='relu')
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)

    if include_top:
        # Classification block
        x = Flatten(name='flatten')(x)
        x = Dense(4096, activation='relu', name='fc1')(x)
        x = Dense(4096, activation='relu', name='fc2')(x)
        x = Dense(classes, activation='softmax', name='predictions')(x)
    else:
       #可以提醒學員為什麼要加avg或是max
        if pooling == 'avg':
            x = GlobalAveragePooling2D()(x)
        elif pooling == 'max':
            x = GlobalMaxPooling2D()(x)

    inputs = img_input

# Create model.
    model = Model(inputs, x, name='VGG16_ResNet_Inception')
    return model
    

model = VGG16_ResNet_Inception(include_top=False)
model.summary()


Model: "VGG16_ResNet_Inception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 224, 224, 1)  0                                            
__________________________________________________________________________________________________
Incpetion_Block-C2a_0_conv (Con (None, 224, 224, 192 192         input_2[0][0]                    
__________________________________________________________________________________________________
Incpetion_Block-C2b_2_conv (Con (None, 224, 224, 192 576         input_2[0][0]                    
__________________________________________________________________________________________________
Incpetion_Block-C2a_0_bn (Batch (None, 224, 224, 192 576         Incpetion_Block-C2a_0_conv[0][0] 
____________________________________________________________________________